# Importing the Keras libraries and packages

In [13]:
from keras.layers import Convolution2D
from keras.models import Sequential
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense , Dropout
import os
import PIL
import cv2

In [12]:
os.environ["CUDA_VISIBLE_DEVICES"] = "GPU-47357fb5-69ce-8a99-7c6f-1c7361564861"
sz = 128

# Step 1 - Building the CNN

In [13]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), input_shape=(sz, sz, 1), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolution layer and pooling
classifier.add(Convolution2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=96, activation='relu'))
classifier.add(Dropout(0.40))
classifier.add(Dense(units=64, activation='relu'))
classifier.add(Dense(units=27, activation='softmax')) # softmax for more than 2

In [14]:
# Compiling the CNN
classifier.compile(optimizer='adam', 
                loss='categorical_crossentropy',
                metrics=['accuracy']) # categorical_crossentropy for more than 2

classifier.summary() # print the model summary

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 126, 126, 32)      320       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 61, 61, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 30, 30, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 28800)             0         
                                                                 
 dense_4 (Dense)             (None, 128)              

# Step 2 - Preparing the train/test data and training the model

In [15]:
import tensorflow as tf

# Define the path to the data directory and the image size
data_dir = 'data_alpha_1'
img_size = (sz, sz)

# Define the batch size and number of classes
batch_size = 10
num_classes = 2

# Define the preprocessing layers
preprocessing_layers = tf.keras.Sequential([
    tf.keras.layers.Rescaling(scale=1./255),
])

# Load the training data
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(data_dir, 'train'),
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='training',
    interpolation='bilinear',
    follow_links=True
)

# Apply the preprocessing layers to the training dataset
train_ds = train_ds.map(lambda x, y: (preprocessing_layers(x), y))

# Load the validation data
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(data_dir, 'train'),
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=img_size,
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset='validation',
    interpolation='bilinear',
    follow_links=True
)

# Apply the preprocessing layers to the validation dataset
val_ds = val_ds.map(lambda x, y: (preprocessing_layers(x), y))

# Load the test data
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=os.path.join(data_dir, 'test'),
    labels='inferred',
    label_mode='categorical',
    color_mode='grayscale',
    batch_size=batch_size,
    image_size=img_size,
    shuffle=False,
    interpolation='bilinear',
    follow_links=True
)

# Apply the preprocessing layers to the test dataset
test_ds = test_ds.map(lambda x, y: (preprocessing_layers(x), y))


# Train the model
classifier.fit(train_ds, epochs=5, validation_data=val_ds)

# Evaluate the model on the test data
test_loss, test_acc = classifier.evaluate(test_ds)
print('Test accuracy:', test_acc)


Found 3724 files belonging to 11 classes.
Using 2980 files for training.
Found 3724 files belonging to 11 classes.
Using 744 files for validation.
Found 1235 files belonging to 11 classes.
Epoch 1/5
298/298 [==============================] - 33s 107ms/step - loss: 1.1942 - accuracy: 0.6295 - val_loss: 0.1325 - val_accuracy: 0.9960
Epoch 2/5
298/298 [==============================] - 31s 105ms/step - loss: 0.1826 - accuracy: 0.9530 - val_loss: 0.0067 - val_accuracy: 0.9987
Epoch 3/5
298/298 [==============================] - 31s 105ms/step - loss: 0.0899 - accuracy: 0.9698 - val_loss: 0.0052 - val_accuracy: 0.9987
Epoch 4/5
298/298 [==============================] - 31s 105ms/step - loss: 0.0616 - accuracy: 0.9815 - val_loss: 3.9435e-04 - val_accuracy: 1.0000
Epoch 5/5
124/124 [==============================] - 4s 28ms/step - loss: 0.2697 - accuracy: 0.9547
Test accuracy: 0.9546558856964111


In [16]:
# Saving the model
model_json = classifier.to_json()
with open("model-bw-alpha_1.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model-bw-alpha_1.h5')
print('Weights saved')

Model Saved
Weights saved
